In [1]:
import plotly
import json
import pathlib
import polars as pl 
import plotly.express as px

In [2]:
llamas = pl.concat([
    pl.from_records(json.load(open('result/2023_12_15_12_05_57_llama_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b@vllm-0.2.5')),
    pl.from_records(json.load(open('result/2023_12_15_12_32_55_llama_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b-awq@vllm-0.2.5')),
    pl.from_records(json.load(open('result/2023_12_15_13_10_47_llama_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b-gptq@vllm-gptq branch')),
    pl.from_records(json.load(open('result/2023_12_15_14_14_12_llama_tgi.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b@tgi-1.3.3')),
    pl.from_records(json.load(open('result/2023_12_15_14_56_18_llama_tgi.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b-eetq@tgi-1.3.3')),
    pl.from_records(json.load(open('result/2023_12_15_15_17_00_llama_tgi.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b-awq@tgi-1.3.3')),
    pl.from_records(json.load(open('result/2023_12_15_15_39_43_llama_tgi.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('llama-7b-gptq@tgi-1.3.3')),
]).to_pandas()
llamas

,batch,total_token,token_per_s,avg_first_token_latency,min_token_latency,max_token_latency,median_token_latency,avg_token_latency,type
0,1,1980,33.000000,0.058836,0.000022,0.051738,0.030188,0.030079,llama-7b@vllm-0.2.5
1,2,3717,61.950000,0.038135,0.000022,0.033544,0.032201,0.032077,llama-7b@vllm-0.2.5
2,3,5522,92.033333,0.045159,0.000020,0.066875,0.032459,0.032353,llama-7b@vllm-0.2.5
3,4,7276,121.266667,0.064571,0.000022,0.070608,0.032745,0.032674,llama-7b@vllm-0.2.5
4,5,8985,149.750000,0.068814,0.000021,0.070883,0.033126,0.033049,llama-7b@vllm-0.2.5
...,...,...,...,...,...,...,...,...,...
891,124,33645,560.750000,11.422629,0.000680,15.003589,0.070776,0.158934,llama-7b-gptq@tgi-1.3.3
892,125,33540,559.000000,11.397340,0.000767,15.002494,0.071026,0.160884,llama-7b-gptq@tgi-1.3.3
893,126,33340,555.666667,11.419920,0.000732,15.001899,0.071503,0.162987,llama-7b-gptq@tgi-1.3.3
894,127,33109,551.816667,11.557860,0.000592,15.004105,0.071722,0.164875,llama-7b-gptq@tgi-1.3.3


In [7]:
fig_llama_throughput = px.line(llamas, x='batch', y='token_per_s', color='type', title='Llama Throughput')
fig_llama_throughput.write_image('images/llama2-7b-throughput.png', scale=2, width=1440)
fig_llama_throughput.show()

In [8]:
fig_llama_avg_latency = px.line(llamas, x='batch', y='avg_token_latency', color='type', title='Llama Average none-first Token Latency')
fig_llama_avg_latency.write_image('images/llama2-7b-avglat.png', scale=2,width=1440)
fig_llama_avg_latency.show()

In [9]:
fig_llama_avg_latency = px.line(llamas, x='batch', y='avg_first_token_latency', color='type', title='Llama Average First Token Latency')
fig_llama_avg_latency.write_image('images/llama2-7b-first-token-lat.png', scale=2, width=1440)
fig_llama_avg_latency.show()

In [5]:
qwen =  pl.concat([
    pl.from_records(json.load(open('result/2023_12_14_23_59_00_qwen_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('qwen-7b-awq@vllm-0.2.5')),
    pl.from_records(json.load(open('result/2023_12_16_17_55_20_qwen_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('qwen-7b@vllm-0.2.5')),
    pl.from_records(json.load(open('result/2023_12_17_01_14_42_qwen_vllm.json'))['results']).drop('raw_ticks').sort('batch').with_columns(type=pl.lit('qwen-7b-int4@vllm-2acd76f')),
])
titles = {
    'token_per_s': 'Throughput',
    'avg_token_latency': 'Average non-first Token Latency',
    'avg_first_token_latency': 'Average First Token Latency',
}

for i, title in titles.items():
    fig = px.line(qwen, x='batch', y=i, color='type', title=f'Qwen 7B {title}')
    fig.write_image(f'images/qwen-7b-{i}.png', scale=2, width=1440)
    fig.show()